<center>
    <img src="https://gitlab.com/ibm/skills-network/courses/placeholder101/-/raw/master/labs/module%201/images/IDSNlogo.png" width="300" alt="cognitiveclass.ai logo"  />
</center>


# **Extract Transform Load (ETL) Lab**


Estimated time needed: **30** minutes


## Objectives

After completing this lab you will be able to:

*   Read CSV and JSON file types.
*   Extract data from the above file types.
*   Transform data.
*   Save the transformed data in a ready-to-load format which data engineers can use to load into an RDBMS.


Import the required modules and functions


In [1]:
import glob                         # this module helps in selecting files 
import pandas as pd                 # this module helps in processing CSV files
import xml.etree.ElementTree as ET  # this module helps in processing XML files.
from datetime import datetime

## Download Files


In [2]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/source.zip

'wget' is not recognized as an internal or external command,
operable program or batch file.


## Unzip Files


In [3]:
!unzip source.zip

'unzip' is not recognized as an internal or external command,
operable program or batch file.


## Set Paths


In [4]:
tmpfile    = "temp.tmp"               # file used to store all extracted data
logfile    = "logfile.txt"            # all event logs will be stored in this file
targetfile = "transformed_data.csv"   # file where transformed data is stored

## Extract


### CSV Extract Function


In [5]:
def extract_from_csv(file_to_process):
    dataframe = pd.read_csv(file_to_process)
    return dataframe

### JSON Extract Function


In [6]:
def extract_from_json(file_to_process):
    dataframe = pd.read_json(file_to_process,lines=True)
    return dataframe

### XML Extract Function


In [7]:
def extract_from_xml(file_to_process):
    dataframe = pd.DataFrame(columns=["name", "height", "weight"])
    tree = ET.parse(file_to_process)
    root = tree.getroot()
    for person in root:
        name = person.find("name").text
        height = float(person.find("height").text)
        weight = float(person.find("weight").text)
        dataframe = dataframe.append({"name":name, "height":height, "weight":weight}, ignore_index=True)
    return dataframe

### Extract Function


In [8]:
def extract():
    extracted_data = pd.DataFrame(columns=['name','height','weight']) # create an empty data frame to hold extracted data
    
    #process all csv files
    for csvfile in glob.glob("*.csv"):
        extracted_data = extracted_data.append(extract_from_csv(csvfile), ignore_index=True)
        
    #process all json files
    for jsonfile in glob.glob("*.json"):
        extracted_data = extracted_data.append(extract_from_json(jsonfile), ignore_index=True)
    
    #process all xml files
    for xmlfile in glob.glob("*.xml"):
        extracted_data = extracted_data.append(extract_from_xml(xmlfile), ignore_index=True)
        
    return extracted_data

## Transform


The transform function does the following tasks.

1.  Convert height which is in inches to millimeter
2.  Convert weight which is in pounds to kilograms


In [9]:
def transform(data):
        #Convert height which is in inches to millimeter
        #Convert the datatype of the column into float
        data.height = data.height.astype(float)
        #Convert inches to meters and round off to two decimals(one inch is 0.0254 meters)
        data['height'] = round(data.height * 0.0254,2)
        
        #Convert weight which is in pounds to kilograms
        #Convert the datatype of the column into float
        data.weight = data.weight.astype(float)
        #Convert pounds to kilograms and round off to two decimals(one pound is 0.45359237 kilograms)
        data['weight'] = round(data.weight * 0.45359237,2)
        return data

## Loading


In [10]:
def load(targetfile,data_to_load):
    data_to_load.to_csv(targetfile)  

## Logging


In [11]:
def log(message):
    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second
    now = datetime.now() # get current timestamp
    timestamp = now.strftime(timestamp_format)
    with open("logfile.txt","a") as f:
        f.write(timestamp + ',' + message + '\n')

## Running ETL Process


In [12]:
log("ETL Job Started")

In [13]:
log("Extract phase Started")
extracted_data = extract()
log("Extract phase Ended")
extracted_data

C:\Users\filiz\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3418: DtypeWarning: Columns (11,13,16,19,20,27,29,30,34,36,37) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,name,height,weight,Unnamed: 0,TV,radio,newspaper,sales,price,mileage,...,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Unnamed: 0.1
0,NaN,NaN,NaN,1.0,230.1,37.8,69.2,22.1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,2.0,44.5,39.3,45.1,10.4,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,3.0,17.2,45.9,69.3,9.3,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,4.0,151.5,41.3,58.5,18.5,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,5.0,180.8,10.8,58.4,12.9,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6777033,NaN,NaN,NaN,3388514.0,NaN,NaN,NaN,NaN,NaN,NaN,...,"Montvila, Rev. Juozas",male,27.0,0.0,0.0,211536,13.00,NaN,S,NaN
6777034,NaN,NaN,NaN,3388515.0,NaN,NaN,NaN,NaN,NaN,NaN,...,"Graham, Miss. Margaret Edith",female,19.0,0.0,0.0,112053,30.00,B42,S,NaN
6777035,NaN,NaN,NaN,3388516.0,NaN,NaN,NaN,NaN,NaN,NaN,...,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1.0,2.0,W./C. 6607,23.45,NaN,S,NaN
6777036,NaN,NaN,NaN,3388517.0,NaN,NaN,NaN,NaN,NaN,NaN,...,"Behr, Mr. Karl Howell",male,26.0,0.0,0.0,111369,30.00,C148,C,NaN


In [14]:
log("Transform phase Started")
transformed_data = transform(extracted_data)
log("Transform phase Ended")
transformed_data 

,name,height,weight,Unnamed: 0,TV,radio,newspaper,sales,price,mileage,...,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Unnamed: 0.1
0,NaN,NaN,NaN,1.0,230.1,37.8,69.2,22.1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,2.0,44.5,39.3,45.1,10.4,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,3.0,17.2,45.9,69.3,9.3,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,4.0,151.5,41.3,58.5,18.5,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,5.0,180.8,10.8,58.4,12.9,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6777033,NaN,NaN,NaN,3388514.0,NaN,NaN,NaN,NaN,NaN,NaN,...,"Montvila, Rev. Juozas",male,27.0,0.0,0.0,211536,13.00,NaN,S,NaN
6777034,NaN,NaN,NaN,3388515.0,NaN,NaN,NaN,NaN,NaN,NaN,...,"Graham, Miss. Margaret Edith",female,19.0,0.0,0.0,112053,30.00,B42,S,NaN
6777035,NaN,NaN,NaN,3388516.0,NaN,NaN,NaN,NaN,NaN,NaN,...,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1.0,2.0,W./C. 6607,23.45,NaN,S,NaN
6777036,NaN,NaN,NaN,3388517.0,NaN,NaN,NaN,NaN,NaN,NaN,...,"Behr, Mr. Karl Howell",male,26.0,0.0,0.0,111369,30.00,C148,C,NaN


In [15]:
log("Load phase Started")
load(targetfile,transformed_data)
log("Load phase Ended")

In [16]:
log("ETL Job Ended")

# Exercise


Using the example above complete the exercise below.


## Download Files


In [17]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/datasource.zip

'wget' is not recognized as an internal or external command,
operable program or batch file.


## Unzip Files


In [18]:
!unzip datasource.zip -d dealership_data

'unzip' is not recognized as an internal or external command,
operable program or batch file.


## About the Data


The file `dealership_data` contains CSV, JSON, and XML files for used car data which contain features named `car_model`, `year_of_manufacture`, `price`, and `fuel`.


## Set Paths


In [19]:
tmpfile    = "dealership_temp.tmp"               # file used to store all extracted data
logfile    = "dealership_logfile.txt"            # all event logs will be stored in this file
targetfile = "dealership_transformed_data.csv"   # file where transformed data is stored

## Extract


### Question 1: CSV Extract Function


In [28]:
# Add the CSV extract function below
def extract_from_csv(file_to_process):
    dataframe = pd.read_csv(file_to_process)
    return dataframe

<details><summary>Click here for the solution</summary>

```
    
def extract_from_csv(file_to_process):
    dataframe = pd.read_csv(file_to_process)
    return dataframe
```

</details>


### Question 2: JSON Extract Function


In [30]:
# Add the JSON extract function below
def extract_from_json(file_to_process):
    dataframe = pd.read_json(file_to_process, lines = True)
    return dataframe

<details><summary>Click here for the solution</summary>

```
    
def extract_from_json(file_to_process):
    dataframe = pd.read_json(file_to_process,lines=True)
    return dataframe
```

</details>


### Question 3: XML Extract Function


In [31]:
# Add the XML extract function below, it is the same as the xml extract function above but the column names need to be renamed.
def extract_from_xml(file_to_process):
    dataframe = pd.DataFrame(columns = ["car_model", "year_of_manufacture", "price", "fuel"])
    tree = ET.parse(file_to_process)
    root = tree.getroot()
    for person in root:
        car_model = person.find("car_model").text
        year_of_manufacture = int(person.find("year_of_manufacture").text)
        price = float(person.find("price").text)
        fuel = person.find("fuel").text
        dataframe = dataframe.append({"car_model" : car_model,
                                     "year_of_manufacture" : year_of_manufacture,
                                     "price" : price,
                                     "fuel" : fuel}, ignore_index = True)
        return dataframe

<details><summary>Click here for the solution</summary>

```
    
def extract_from_xml(file_to_process):
    dataframe = pd.DataFrame(columns=['car_model','year_of_manufacture','price', 'fuel'])
    tree = ET.parse(file_to_process)
    root = tree.getroot()
    for person in root:
        car_model = person.find("car_model").text
        year_of_manufacture = int(person.find("year_of_manufacture").text)
        price = float(person.find("price").text)
        fuel = person.find("fuel").text
        dataframe = dataframe.append({"car_model":car_model, "year_of_manufacture":year_of_manufacture, "price":price, "fuel":fuel}, ignore_index=True)
    return dataframe
```

</details>


### Question 4: Extract Function

Call the specific extract functions you created above by replacing the `ADD_FUNCTION_CALL` with the proper function call.


In [34]:
def extract():
    extracted_data = pd.DataFrame(columns=['car_model','year_of_manufacture','price', 'fuel']) # create an empty data frame to hold extracted data
    
    #process all csv files
    for csvfile in glob.glob("dealership_data/*.csv"):
        extracted_data = extracted_data.append(extarcted_from_csv(csvfile), ignore_index=True)
        
    #process all json files
    for jsonfile in glob.glob("dealership_data/*.json"):
        extracted_data = extracted_data.append(extarcted_from_json(jsonfile), ignore_index=True)
    
    #process all xml files
    for xmlfile in glob.glob("dealership_data/*.xml"):
        extracted_data = extracted_data.append(extarcted_from_xml(xmlfile), ignore_index=True)
        
    return extracted_data

<details><summary>Click here for the solution</summary>

```
    
def extract():
    extracted_data = pd.DataFrame(columns=['car_model','year_of_manufacture','price', 'fuel']) # create an empty data frame to hold extracted data
    
    #process all csv files
    for csvfile in glob.glob("dealership_data/*.csv"):
        extracted_data = extracted_data.append(extract_from_csv(csvfile), ignore_index=True)
        
    #process all json files
    for jsonfile in glob.glob("dealership_data/*.json"):
        extracted_data = extracted_data.append(extract_from_json(jsonfile), ignore_index=True)
    
    #process all xml files
    for xmlfile in glob.glob("dealership_data/*.xml"):
        extracted_data = extracted_data.append(extract_from_xml(xmlfile), ignore_index=True)
        
    return extracted_data
```

</details>


## Transform


### Question 5: Transform

Round the `price` columns to 2 decimal places


In [35]:
# Add the transform function below
def transform(data):
    data["price"] = round(data.price, 2)
    return data

<details><summary>Click here for the solution</summary>

```

def transform(data):
        data['price'] = round(data.price, 2)
        return data
```

</details>


## Loading


### Question 6: Load


In [36]:
# Add the load function below
def load(targetfile, data_to_load):
    data_to_load.to_csv(targetfile)

<details><summary>Click here for the solution</summary>

```

def load(targetfile,data_to_load):
    data_to_load.to_csv(targetfile)  
```

</details>


## Logging


### Question 7: Log

Make sure to change the name of the logfile to the one specified in the set paths section. Change the timestamp order to Hour-Minute-Second-Monthname-Day-Year.


In [37]:
# Add the log function below
def log(message):
    timestamp_format = "%H:%M:%S-%h-%d-%Y"
    now = datetime.now()
    timestamp = now.strftime(timestamp_format)
    with open("dealership_logfile.txt", "a") as f:
        f.write(timestamp + "," + message + "\n")

<details><summary>Click here for the solution</summary>

```

def log(message):
    timestamp_format = '%H:%M:%S-%h-%d-%Y' #Hour-Minute-Second-MonthName-Day-Year
    now = datetime.now() # get current timestamp
    timestamp = now.strftime(timestamp_format)
    with open("dealership_logfile.txt","a") as f:
        f.write(timestamp + ',' + message + '\n') 
```

</details>


## Running ETL Process


### Question 8: ETL Process

Run all functions to extract, transform, and load the data. Make sure to log all events using the `log` function. Place your code under each comment.


In [38]:
# Log that you have started the ETL process
log("ETL Job Started")

# Log that you have started the Extract step
log("Extract phase Started")
# Call the Extract function
extracted_data = extract()
# Log that you have completed the Extract step

log("Extract phase Ended")
# Log that you have started the Transform step
log("Transform phase Started")
# Call the Transform function
transformed_data = transform(extracted_data)
# Log that you have completed the Transform step
log("Transform phase Ended")

# Log that you have started the Load step
log("Load phase Started")
# Call the Load function
load(targetfile, transformed_data)
# Log that you have completed the Load step
log("Load phase Ended")

# Log that you have completed the ETL process
log("ETL Job Ended")

<details><summary>Click here for the solution</summary>

```

log("ETL Job Started")

log("Extract phase Started")
extracted_data = extract()
log("Extract phase Ended")

log("Transform phase Started")
transformed_data = transform(extracted_data)
log("Transform phase Ended")

log("Load phase Started")
load(targetfile,transformed_data)
log("Load phase Ended")

log("ETL Job Ended")
```

</details>


## Authors


Ramesh Sannareddy

Joseph Santarcangelo

Azim Hirjani


## Change Log


| Date (YYYY-MM-DD) | Version | Changed By        | Change Description                 |
| ----------------- | ------- | ----------------- | ---------------------------------- |
| 2020-11-25        | 0.1     | Ramesh Sannareddy | Created initial version of the lab |


Copyright © 2020 IBM Corporation. This notebook and its source code are released under the terms of the [MIT License](https://cognitiveclass.ai/mit-license?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkPY0221ENSkillsNetwork23455645-2021-01-01&cm_mmc=Email_Newsletter-\_-Developer_Ed%2BTech-\_-WW_WW-\_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork-23455645&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ).
